In [420]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
import os
import urllib
url = "https://covid19.who.int/WHO-COVID-19-global-data.csv"
file_path = os.path.join("data", "covid")
os.makedirs(file_path, exist_ok=True)
csv_path = os.path.join(file_path, "WHO-COVID-19-global-data.csv")
urllib.request.urlretrieve(url, csv_path)
df = pd.read_csv(csv_path)
df2 = pd.read_csv("data/World-GDP.csv")
df.columns = [col.strip() for col in df.columns]
df['Date_reported'] = pd.to_datetime(df["Date_reported"])
isO = df['Date_reported']==pd.to_datetime('2021-02-21 00:00:00')
df = df[isO]
df = df.reset_index()
df.drop(columns=["New_deaths", "New_cases", "index", 'WHO_region', "Country_code", "Date_reported"], inplace=True)
df.rename(columns={'Country': "Area"}, inplace=True)
df2.rename(columns={'Country or Area': "Area"}, inplace=True)
df = df[df.Cumulative_deaths > 0]
df = df[df.Cumulative_cases > 20000]
df2.drop(columns=["Year", "Item"], inplace=True)
df2.drop([201, 202, 40, 41, 42], inplace=True)
df2 = df2.replace({"Iran, Islamic Republic of":"Iran (Islamic Republic of)","Kingdom of Eswatini":"Eswatini", "Republic of North Macedonia":"North Macedonia", "Sint Maarten (Dutch part)":"Sint Maarten", "United Kingdom of Great Britain and Northern Ireland":"The United Kingdom", "Kosovo":"Kosovo[1]","United States":"United States of America","State of Palestine":"occupied Palestinian territory, including east Jerusalem"})
df2 = df2.replace({"Côte d'Ivoire":"Côte d’Ivoire"})
dfT = pd.DataFrame([['United Republic of Tanzania', 81122.12]], columns=["Area", "Value"])
dfT2 = pd.DataFrame([['China', 10261.68]], columns=["Area", "Value"])
df2 = df2.append(dfT, ignore_index=True)
df2 = df2.append(dfT2, ignore_index=True)
df.set_index("Area", inplace=True)
df2.set_index("Area", inplace=True)
merged = pd.merge(df, df2, how="inner", left_index=True, right_index=True)
merged["rate"] = merged.apply(lambda x: (x["Cumulative_deaths"]/x["Cumulative_cases"])*100, axis=1)
print(merged[merged["Value"]<5000]["rate"].mean())
print(merged[merged["Value"]>30000]["rate"].mean())
print(stats.pearsonr(merged["Value"],merged["rate"])[0])
sortedM = merged.sort_values(by=['rate'])
sortedM


2.1468405591145143
1.7014203265441386
-0.18872088655093652


,Cumulative_cases,Cumulative_deaths,Value,rate
Area,,,,
Singapore,59858,29,64102.737611,0.048448
Qatar,159967,256,64781.733197,0.160033
United Arab Emirates,368175,1108,43103.323058,0.300944
Thailand,25415,83,7784.739080,0.326579
Bahrain,117234,420,23503.977127,0.358258
...,...,...,...,...
China,101669,4842,10261.680000,4.762514
Ecuador,273097,15513,6183.823825,5.680399
Egypt,177543,10298,3161.324684,5.800285


In [422]:
print(stats.pearsonr(merged["Cumulative_cases"],merged["rate"])[0])

0.03483030132050796
